## Libraries for TSA using AutoTS

In [6]:
import pandas as pd
from autots import AutoTS
import matplotlib.pyplot as plt
%matplotlib inline

## Passengers data for TSA

In [7]:
df = pd.read_csv('airline_passengers.csv')
df

,Month,Thousands of Passengers
0,1949-01,112.0
1,1949-02,118.0
2,1949-03,132.0
3,1949-04,129.0
4,1949-05,121.0
...,...,...
141,1960-10,461.0
142,1960-11,390.0
143,1960-12,432.0
144,NaN,NaN


### Dropping NA values

In [8]:
df.dropna(axis=0, inplace=True)
df.tail()

,Month,Thousands of Passengers
139,1960-08,606.0
140,1960-09,508.0
141,1960-10,461.0
142,1960-11,390.0
143,1960-12,432.0


### Changing date column dtype: `object` to `datetime`

In [9]:
df['Month'] = pd.to_datetime(df['Month'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 144 entries, 0 to 143
Data columns (total 2 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Month                    144 non-null    datetime64[ns]
 1   Thousands of Passengers  144 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 3.4 KB


In [11]:
from greykite.framework.templates.autogen.forecast_config import ForecastConfig
from greykite.framework.templates.autogen.forecast_config import MetadataParam
from greykite.framework.templates.forecaster import Forecaster 
from greykite.framework.templates.model_templates import ModelTemplateEnum
from greykite.framework.utils.result_summary import summarize_grid_search_results
# Specifies dataset information
metadata = MetadataParam(
     time_col="Month",  # name of the time column
     value_col="Thousands of Passengers",  # name of the value column
     freq="MS"  #"MS" for Montly at start date
 )
forecaster = Forecaster()
result = forecaster.run_forecast_config(
     df=df,
     config=ForecastConfig(
         model_template=ModelTemplateEnum.SILVERKITE.name,
         forecast_horizon=100,  # forecasts 100 steps ahead
         coverage=0.95,  # 95% prediction intervals
         metadata_param=metadata
    )
)

ModuleNotFoundError: No module named 'greykite'

In [ ]:
ts = result.timeseries
fig = ts.plot()
plotly.io.show(fig)

In [13]:
import greykite

ModuleNotFoundError: No module named 'greykite'